##### 动态规划

##### 一、适用范围
* 搜索中常见的问题：重复搜索

##### 二、问题
* 钢筋切割问题：不同长度价钱不同，怎么保证最大收益？

##### 三、动态规划常见步骤
* 都有一个重复的子问题
* 可以将重复子问题的运算用表存起来
* 将存储结果解析
    

In [142]:
from collections import defaultdict
# 初始化 价格 -> 长度 
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 31]
price = defaultdict(int)
for i, p in enumerate(original_price) : 
    price[i + 1] = p

# 最优价钱
def r(n) : 
    #不断切割，得出所有组合情况，找出最大值     
    return max([price[n]] + [r(n - i) + r(i) for i in range(1, n)])

In [14]:
#  [r(n - i) + r(i) for i in range(1, n)] 这里存在大量重复计算重复切割
r(15)

43

##### 四、装饰器（有点像Java里面的拦截器、AOP）

In [31]:
# 统计调用次数
called_times = defaultdict(int)
def call_times(fun, arg) : 
    print("function {} called once ".format(fun.__name__))
    called_times[fun.__name__] += 1
    return fun(arg)

In [24]:
call_times(r, 5)

function r called once 


13

In [25]:
called_times

defaultdict(int, {'r': 2})

In [35]:
def r(n) : 
    # 统计下计算次数，会发现大量重复计算
    fName = r.__name__
    called_times[(fName, n)] += 1
    return max([price[n]] + [r(n - i) + r(i) for i in range(1, n)])

In [36]:
r(15)

43

In [37]:
called_times

defaultdict(int,
            {('r', 15): 1,
             ('r', 14): 2,
             ('r', 13): 6,
             ('r', 12): 18,
             ('r', 11): 54,
             ('r', 10): 162,
             ('r', 9): 486,
             ('r', 8): 1458,
             ('r', 7): 4374,
             ('r', 6): 13122,
             ('r', 5): 39366,
             ('r', 4): 118098,
             ('r', 3): 354294,
             ('r', 2): 1062882,
             ('r', 1): 3188646})

In [44]:
def call_times(fun) : 
    # 内部函数，并将函数返回    
    def wrap(n) : 
        fName = fun.__name__
        called_times[(fName, n)] += 1
        return fun(n)
    
    return wrap

In [45]:
call_f = call_times(r)

In [52]:
call_f(15)

43

In [53]:
called_times

defaultdict(int,
            {('r', 15): 2,
             ('r', 14): 2,
             ('r', 13): 6,
             ('r', 12): 18,
             ('r', 11): 54,
             ('r', 10): 162,
             ('r', 9): 486,
             ('r', 8): 1458,
             ('r', 7): 4374,
             ('r', 6): 13122,
             ('r', 5): 39366,
             ('r', 4): 118102,
             ('r', 3): 354294,
             ('r', 2): 1062882,
             ('r', 1): 3188646})

In [147]:
from functools import wraps
called_times = defaultdict(int)

def call_times(fun) : 
    # 内部函数，并将函数返回 
    @wraps(fun)
    def wrap(n) : 
        fName = fun.__name__
        called_times[(fName, n)] += 1
        return fun(n)
    
    return wrap

def memo(fun) : 
    cache = defaultdict(int)
    
    def _wrap(arg) : 
        if not arg in cache : 
            cache[arg] = fun(arg)
        return cache[arg]
    
    return _wrap
    
@call_times   
@memo
def r(n) : 
    return max([price[n]] + [r(n - i) + r(i) for i in range(1, n)])

In [148]:
solution = {}

@call_times   
@memo
def r(n) : 
    max_price, max_split = max([(price[n], 0)] + [(r(n - i) + r(i), i) for i in range(1, n)], key=lambda x: x[0])
    solution[n] = (n - max_split, max_split)
    return max_price

In [149]:
r(19)

55

In [110]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (10, 2),
 13: (10, 3),
 14: (12, 2),
 15: (13, 2)}

In [125]:
def parse_solution(n) : 
    left, right = solution[n]
    
    if right == 0 : return [left]
    return parse_solution(left) + parse_solution(right)

In [151]:
parse_solution(19)

[10, 6, 3]

In [268]:
solution = {}
# @lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    # 递归结束条件     
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    # 候选操作？对目标结果做两种情况的推导，s1 去除一部分 或 s2去除一部分
    # 对应推到操作，s1 删除元素 或 s1 添加元素
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    # 一般性操作？尾部元素一致，则不进行任何操作，继续后续推到
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [262]:
import re
def parse_solution(s1, s2) : 
    ops = []
    if len(s1) == 0 or len(s2) == 0 or step == 0 : return ops
    ops = ops + parse_solution(s1[:-1], s2)
    ops = ops + parse_solution(s1, s2[:-1])
    can_try = [(k, v) for k, v in solution.items() if k[0] == s1]
    
    for (k, v) in can_try : 
        
    

In [269]:
edit_distance('AB', 'ACD')

2

In [270]:
solution

{('A', 'A'): '',
 ('A', 'AC'): 'ADD C',
 ('A', 'ACD'): 'ADD D',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AC'): 'SUB B => C',
 ('AB', 'ACD'): 'ADD D'}

In [266]:
parse_solution('AB', 'ACD', step)